In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import re
import emoji
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import sys
from unidecode import unidecode
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import to_categorical
from gensim.models.keyedvectors import KeyedVectors
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import RMSprop
from keras.metrics import Precision, Recall
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
!pip install unidecode
!pip install emoji
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 8.7 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True


## 1. Écrire un script python qui permet de faire l’ensemble des pré-traitements.

In [3]:
tknzr = TweetTokenizer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) - set(('not', 'no', 'never', 'nor'))

def standardization(tweet):
    tweet = re.sub(r"\u2019", "'", tweet)
    tweet = re.sub(r"\u002C", ",", tweet)
    tweet = ' '.join(str2emoji(unidecode(tweet).lower().split()))
    tweet = re.sub(r"(http|https)?:\/\/[a-zA-Z0-9\-.]+\.[a-zA-Z]{2,4}(/\S*)?", " ", tweet)
    tweet = re.sub(r"\bve\b", "have", tweet)  # Correction: remove extra 're' in 've'
    tweet = re.sub(r"\bcan't\b", "cannot", tweet)
    tweet = re.sub(r"\b'n't\b", " not", tweet)
    tweet = re.sub(r"\bre\b", "are", tweet)  # Correction: remove extra 're' in 'are'
    tweet = re.sub(r"\bd\b", "would", tweet)  # Correction: remove extra 'd' in 'would'
    tweet = re.sub(r"\b'll\b", "will", tweet)
    tweet = re.sub(r"\b's\b", " ", tweet)
    tweet = re.sub(r"\bn\b", " ", tweet)
    tweet = re.sub(r"\bm\b", "am", tweet)
    tweet = re.sub(r"@\w+", " ", tweet)
    tweet = re.sub(r"#\w+", " ", tweet)
    tweet = re.sub(r"[0-9]+", " ", tweet)
    tweet = tknzr.tokenize(tweet)
    tweet = [lemmatizer.lemmatize(i, j[0].lower()) if j[0].lower() in ['a','n','v'] else lemmatizer.lemmatize(i) for i, j in pos_tag(tweet)]
    tweet = [i for i in tweet if (i.lower() not in stop_words) and (i.lower() not in punctuation)]
    tweet = ' '.join(tweet)
    return tweet

def str2emoji(tweet):
    for pos,ej in enumerate(tweet):
        if ej in emojis:
            tweet[pos]=emojis[ej]
    return tweet

"""
Emoticons and Emoji data dictionary
"""
emojis = {
    u":)": "🙂",
    u":-)": "🙂",
    u":]": "🙂",
    u":-3": "🙂",
    u":3": "🙂",
    u":->": "🙂",
    u":>": "🙂",
    u"8-)": "🙂",
    u":o)": "🙂",
    u":-}": "🙂",
    u":}": "🙂",
    u":-)": "🙂",
    u":c)": "🙂",
    u":^)": "🙂",
    u"=]": "🙂",
    u"=)": "🙂",
    u":(": "🙁",
    u":-(": "🙁",
    u":[": "🙁",
    u">:[": "🙁",
    u":{": "🙁",
    u">:(": "🙁",
    u":'(": "😢",
    u":'-(": "😢",
    u"D:": "😨",
    u":o": "😮",
    u":-o": "😮",
    u"o_O": "😮",
    u"o.O": "😮",
    u"o_o": "😮",
    u"0_o": "😮",
    u"T_T": "😭",
    u">_<": "😣",
    u"O_O": "😳",
    u"^_^": "😊",
    u"(^_^)": "😊",
    u"(-_-)": "😑",
    u"(._.)": "😐",
    u"(>_<)": "😣",
    u"(^o^)": "😃",
    u"(^O^)": "😃",
    u"(^0^)": "😃",
    u"(@_@)": "😵",
    u"(+o+)": "😵",
    u"(*_*)": "😲",
    u"(^_^)/": "😊",
    u"(>o<)": "😣",
    u"(-_\\": "😑",
    u"(~_~)": "😑",
    u"(-.-)zzz": "😴",
    u"(^_-)": "😉",
    u"((+_+))": "😵",
    u"(+o+)": "😵",
    u"(-)__(-)": "😑",
}

def data_preprocessing(path_tweets):
    data = pd.read_csv(path_tweets, encoding='utf-8', sep='\t')
    data['tweet'] = data['Tweet'].apply(lambda x: standardization(x))
    data['label'] = data['Intensity Class'].str.split(":").str[0]
    return data['tweet'], data['label']

In [4]:
df_3 = pd.read_csv("twitter_training.csv", delimiter=',', names=['ID', 'Game', 'label', 'tweet'])
df_7 = "2018-Valence-oc-En-train.txt"

df_3.drop(columns=['ID'], inplace=True)
df_3.drop(columns=['Game'], inplace=True)

# Supprimer les lignes avec des valeurs NaN dans toutes les colonnes
df_3.dropna(inplace=True)

# Supprimer les lignes où la colonne 'labels' a la valeur 'Irrelevant'
df_3 = df_3[df_3['label'] != 'Irrelevant']


# Mapping des valeurs de la colonne 'labels' vers les valeurs numériques correspondantes
label_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}


# Remplacer les valeurs dans la colonne 'labels' par les valeurs numériques correspondantes
df_3.loc[:, 'label'] = df_3['label'].replace(label_mapping)

tweets_train_3 = df_3['tweet'].apply(lambda x: standardization(x))
sentiments_train_3 = df_3['label']

tweets_train_7, sentiments_train_7 = data_preprocessing(df_7)
all_tweets = pd.concat([tweets_train_3, tweets_train_7], ignore_index=True)

In [5]:
tweets_train_3

0                                 im get borderland murder
1                                         come border kill
2                                   im get borderland kill
3                                im come borderland murder
4                                 im get borderland murder
                               ...                        
74677    realize window partition mac like year behind ...
74678    realize mac window partition year behind nvidi...
74679    realize window partition mac year behind nvidi...
74680    realize window partition mac like year behind ...
74681    like window partition mac like year behind dri...
Name: tweet, Length: 61121, dtype: object

In [6]:
tweets_train_7

0                                        yeah 🙂 play well
1       least guy try discourage anymore want never be...
2       uplift you'are still discourage mean you'are l...
3                           ... age heyday blood tame ...
4       embarrassed saw u like knvfkkjg think we'are s...
                              ...                        
1176    think ian knew pavel know charlie bet rob cack...
1177                              lose wallet lol ... ...
1178    repentance trust christ lowly painful also joy...
1179                                                 chew
1180                      anxious time leave house horrid
Name: tweet, Length: 1181, dtype: object

In [7]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
tokenizer.fit_on_texts(all_tweets)

sequences_train_3 = tokenizer.texts_to_sequences(tweets_train_3)
sequences_train_7 = tokenizer.texts_to_sequences(tweets_train_7)

print(tweets_train_3[0])
print(sequences_train_3[0])

MAX_SEQUENCE_LENGTH = 0
for elt in all_tweets:
    if len(elt) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(elt)

data_train_3 = pad_sequences(sequences_train_3, maxlen=MAX_SEQUENCE_LENGTH)
data_train_7 = pad_sequences(sequences_train_7, maxlen=MAX_SEQUENCE_LENGTH)

print(MAX_SEQUENCE_LENGTH)
print(data_train_3[0])

im get borderland murder
[213, 3, 96, 1290]
799
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    

In [8]:
print(data_train_3.shape[0])
print(data_train_3[-10:])
indices_train_3 = np.arange(data_train_3.shape[0])
print(indices_train_3[-10:])
data_train_3 = data_train_3[indices_train_3]
print(data_train_3[-10:])
indices_train_7 = np.arange(data_train_7.shape[0])
data_train_7 = data_train_7[indices_train_7]

61121
[[   0    0    0 ...  322 6413 9254]
 [   0    0    0 ...   75 9254  261]
 [   0    0    0 ...  322 9254  261]
 ...
 [   0    0    0 ...   19  327  941]
 [   0    0    0 ...  327   77  941]
 [   0    0    0 ...   19  327  941]]
[61111 61112 61113 61114 61115 61116 61117 61118 61119 61120]
[[   0    0    0 ...  322 6413 9254]
 [   0    0    0 ...   75 9254  261]
 [   0    0    0 ...  322 9254  261]
 ...
 [   0    0    0 ...   19  327  941]
 [   0    0    0 ...  327   77  941]
 [   0    0    0 ...   19  327  941]]


In [9]:
#print(labels_train)
labels_train_3 = to_categorical(np.asarray(sentiments_train_3), 3)
labels_train_3 = labels_train_3[indices_train_3]
print(labels_train_3.shape)

labels_train_7 = to_categorical(np.asarray(sentiments_train_7), 7)
labels_train_7 = labels_train_7[indices_train_7]
print(labels_train_7.shape)


(61121, 3)
(1181, 7)


In [10]:
split_idx_7 = int(len(data_train_7)*0.85)
split_idx_3 = int(len(data_train_3)*0.85)
x_train_3, x_val_3 = data_train_3[:split_idx_3], data_train_3[split_idx_3:]
y_train_3, y_val_3 = labels_train_3[:split_idx_3], labels_train_3[split_idx_3:]

x_train_7, x_val_7 = data_train_7[:split_idx_7], data_train_7[split_idx_7:]
y_train_7, y_val_7 = labels_train_7[:split_idx_7], labels_train_7[split_idx_7:]

### 2. Écrire un script qui permet de générer automatiquement la représentation vectorielle des tweets.

In [11]:
word_index = tokenizer.word_index
print(word_index)
nb_words = len(word_index) + 1
EMBEDDING_DIM = 300
oov = []
oov.append((np.random.rand(EMBEDDING_DIM) * 2.0) - 1.0)
oov = oov / np.linalg.norm(oov)

EMBEDDING_FILE = "GoogleNews-vectors-negative300.bin"
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))



{'game': 1, 'com': 2, 'get': 3, 'play': 4, 'not': 5, 'go': 6, 'like': 7, 'pic': 8, "i'am": 9, 'twitter': 10, 'make': 11, 'fuck': 12, 'one': 13, 'time': 14, 'love': 15, 'johnson': 16, 'good': 17, 'new': 18, 'no': 19, 'really': 20, 'u': 21, 'look': 22, 'unk': 23, 'see': 24, 'shit': 25, 'p': 26, 'cannot': 27, 'people': 28, 'still': 29, 'day': 30, 'year': 31, 'amazon': 32, 'say': 33, 'best': 34, 'facebook': 35, 'work': 36, 'k': 37, 'microsoft': 38, 'dead': 39, 'great': 40, 'even': 41, 'would': 42, 'know': 43, 'want': 44, 'xbox': 45, 'come': 46, 'rhandlerr': 47, 'please': 48, 'google': 49, 'red': 50, 'back': 51, 'fix': 52, 'think': 53, 'home': 54, 'take': 55, 'much': 56, 'call': 57, 'try': 58, 'tv': 59, 'buy': 60, 'watch': 61, 'nvidia': 62, 'win': 63, 'verizon': 64, 'wait': 65, 'fifa': 66, 'x': 67, 'first': 68, 'use': 69, "i'have": 70, 'guy': 71, 'v': 72, 'fun': 73, 'today': 74, 'video': 75, 'need': 76, 'ever': 77, 'thing': 78, 'also': 79, 'player': 80, 'bad': 81, 'give': 82, 'help': 83, 't

In [12]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
print(word2vec)
for word, i in word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]
    else:
        embedding_matrix[i] = oov

KeyedVectors<vector_size=300, 3000000 keys>


In [ ]:
word2vec['game']

array([ 1.14746094e-01,  5.27343750e-02,  4.29687500e-02,  4.48608398e-03,
        2.73437500e-01,  7.17773438e-02,  1.73828125e-01, -1.98242188e-01,
        2.07519531e-02,  1.18652344e-01,  3.34472656e-02, -1.05957031e-01,
        3.33786011e-04, -1.79687500e-01, -1.31225586e-02, -1.26953125e-01,
        1.48437500e-01,  1.41601562e-01,  1.55273438e-01, -1.03027344e-01,
        2.17773438e-01,  1.18164062e-01,  9.66796875e-02, -1.51367188e-01,
       -2.07519531e-02, -2.96630859e-02, -7.12890625e-02, -1.63574219e-02,
        1.72851562e-01, -7.75146484e-03, -1.04980469e-01,  1.14746094e-01,
        1.46484375e-01, -1.94335938e-01, -5.78613281e-02,  2.08007812e-01,
        1.70898438e-01, -3.36914062e-02,  9.37500000e-02,  2.35351562e-01,
        2.61718750e-01,  1.56250000e-02,  2.92968750e-01,  1.67968750e-01,
       -5.07812500e-02, -1.59179688e-01,  1.05957031e-01, -1.04003906e-01,
        8.30078125e-02,  2.75390625e-01,  4.02832031e-02,  2.61718750e-01,
        8.23974609e-03, -

In [ ]:
df_3test = pd.read_csv("twitter_test.csv", delimiter=',', names=['ID', 'Game', 'label', 'tweet'])

df_3test.drop(columns=['ID'], inplace=True)

df_3test.drop(columns=['Game'], inplace=True)

# Supprimer les lignes avec des valeurs NaN dans toutes les colonnes
df_3test.dropna(inplace=True)

# Supprimer les lignes où la colonne 'labels' a la valeur 'Irrelevant'
df_3test = df_3[df_3['label'] != 'Irrelevant']


# Mapping des valeurs de la colonne 'labels' vers les valeurs numériques correspondantes
label_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}


# Remplacer les valeurs dans la colonne 'labels' par les valeurs numériques correspondantes
df_3test.loc[:, 'label'] = df_3['label'].replace(label_mapping)

tweets_test_3 = df_3test['tweet'].apply(lambda x: standardization(x))
sentiments_test_3 = df_3test['label']

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
tokenizer.fit_on_texts(all_tweets)

sequences_test_3 = tokenizer.texts_to_sequences(tweets_test_3)


data_test_3 = pad_sequences(sequences_test_3, maxlen=MAX_SEQUENCE_LENGTH)


indices_test_3 = np.arange(data_test_3.shape[0])

data_test_3 = data_test_3[indices_test_3]


labels_test_3 = to_categorical(np.asarray(sentiments_test_3), 3)
labels_test_3 = labels_test_3[indices_test_3]

#### 3. Entraîner un réseau de neurones profond pour la tâche source (i.e classification d’un tweet à l’une de trois classes).

In [ ]:

print("x_train_3 ", x_train_3.shape )
print("y_train_3 ", y_train_3.shape )
print("x_val_3 ", x_val_3.shape )
print("y_val_3 ", y_val_3.shape )
print("==================================================")
print("x_train_7 ", x_train_7.shape )
print("y_train_7 ", y_train_7.shape )
print("x_val_7 ", x_val_7.shape )
print("y_val_7 ", y_val_7.shape )

x_train_3  (51952, 799)
y_train_3  (51952, 3)
x_val_3  (9169, 799)
y_val_3  (9169, 3)
x_train_7  (1003, 799)
y_train_7  (1003, 7)
x_val_7  (178, 799)
y_val_7  (178, 7)


In [ ]:
embedding_layer = Embedding(input_dim=nb_words,
                            output_dim=EMBEDDING_DIM,
                            trainable=False,
                            name='embedding_layer')

model = Sequential()

model.add(embedding_layer)

model.add(Bidirectional(LSTM(64, return_sequences=True),input_shape=(1, MAX_SEQUENCE_LENGTH)))

model.add(Dropout(0.2, name='dropout'))

model.add(Bidirectional(LSTM(64, name='bidirectional_lstm')))

model.add(Dense(3, activation='softmax', name='output_dense'))


# Compilation du modèle
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

# Affichage de la structure du modèle
history = model.fit(x_train_3, y_train_3,
                    validation_data=(x_val_3, y_val_3),
                    batch_size=64, epochs=25)

model.summary()




Epoch 1/25
812/812 [==============================] - 99s 114ms/step - loss: 0.9621 - accuracy: 0.5282 - val_loss: 0.9900 - val_accuracy: 0.5384
Epoch 2/25
812/812 [==============================] - 95s 117ms/step - loss: 0.8610 - accuracy: 0.6082 - val_loss: 0.9704 - val_accuracy: 0.5537
Epoch 3/25
812/812 [==============================] - 90s 111ms/step - loss: 0.8224 - accuracy: 0.6307 - val_loss: 0.9945 - val_accuracy: 0.5503
Epoch 4/25
812/812 [==============================] - 94s 116ms/step - loss: 0.7838 - accuracy: 0.6555 - val_loss: 1.0188 - val_accuracy: 0.5537
Epoch 5/25
812/812 [==============================] - 94s 116ms/step - loss: 0.7355 - accuracy: 0.6801 - val_loss: 1.0621 - val_accuracy: 0.5523
Epoch 6/25
812/812 [==============================] - 94s 116ms/step - loss: 0.6836 - accuracy: 0.7056 - val_loss: 1.0916 - val_accuracy: 0.5393
Epoch 7/25
812/812 [==============================] - 90s 111ms/step - loss: 0.6303 - accuracy: 0.7321 - val_loss: 1.1185 - val_ac

# Evaluation de model 

In [ ]:
# Evaluate model on the test set

accuracy= model.evaluate(data_test_3, labels_test_3 , verbose=0)

print('Accuracy  : {:.4f}'.format(accuracy[1]))


Accuracy  : 0.8852


In [ ]:
model.save("model_3C.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# classification d’un tweet à l’une de sept classes

In [ ]:
# Chargement du modèle entraîné sur 3 classes
model2 = load_model('model_3C.h5')
model2.layers.pop()
model2.layers.pop()

model2.add(Dense(150, activation='relu', name='dense11'))
model2.add(Dense(64, activation='relu', name='dense21'))
model2.add(Dense(7, activation='softmax', name='dense31'))
model2.summary()
model2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])
history = model2.fit(x_train_7, y_train_7, validation_data=(x_val_7, y_val_7), epochs=30, batch_size=50)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding  (None, None, 300)         7072200   
 )                                                               
                                                                 
 bidirectional_2 (Bidirecti  (None, None, 128)         186880    
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 output_dense (Dense)        (None, 3)                 387       
                                                      

# Réaliser les prédictions sur le corpus Test.

In [ ]:
model2.save("model_7C.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model_7 = load_model('model_3C.h5')





# Liste des noms de fichiers à concaténer
file_names = ["2018-EI-oc-En-anger-test-gold.txt",
              "2018-EI-oc-En-fear-test-gold.txt",
              "2018-EI-oc-En-joy-test-gold.txt",
              "2018-EI-oc-En-sadness-test-gold.txt"]

# Lire le premier fichier avec l'en-tête
concatenated_df = pd.read_csv(file_names[0], encoding='utf-8', sep='\t')

# Lire les fichiers suivants sans en-tête et concaténer les données
for file_name in file_names[1:]:
    df = pd.read_csv(file_name, encoding='utf-8', sep='\t', header=None)  # Ignorer l'en-tête
    concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)

# Enregistrer le DataFrame concaténé dans un fichier texte
concatenated_df.to_csv("concatenated_data.txt", encoding='utf-8', sep='\t', index=False)



In [ ]:
tweets_test, sentiments_test = data_preprocessing("concatenated_data.txt")

In [ ]:
def predict_class(text):
    '''Function to predict sentiment class of the passed text'''
    sentiment_classes = sentiments_test
    max_len=50

    # Transforms text to a sequence of integers using a tokenizer object
    xt = tokenizer.texts_to_sequences(text)
    # Pad sequences to the same length
    xt = pad_sequences(xt, padding='post', maxlen=max_len)
    # Do the prediction using the loaded model
    yt = model_7.predict(xt).argmax(axis=1)
    # Print the predicted sentiment
    print('The predicted sentiment is',sentiment_classes[yt[0]] )

In [ ]:
print(tweets_test[0])
print(sentiments_test[0])
predict_class(tweets_test[0])

know mean well i'am offend prick
2
1/1 [==============================] - 0s 24ms/step
The predicted sentiment is 2


In [ ]:
print(tweets_test[1])
print(sentiments_test[1])
predict_class(tweets_test[1])

try free alpha trial watch i'am sell worth subscription alone incredible
1
2/2 [==============================] - 0s 9ms/step
The predicted sentiment is 1


In [ ]:
print(tweets_test[2])
print(sentiments_test[2])
predict_class(tweets_test[2])

people truly day
3
1/1 [==============================] - 0s 25ms/step
The predicted sentiment is 3


In [ ]:
print(tweets_test[3])
print(sentiments_test[3])
predict_class(tweets_test[3])

bully plain simple
1
1/1 [==============================] - 0s 25ms/step
The predicted sentiment is 1


In [ ]:
print(tweets_test[4])
print(sentiments_test[4])
predict_class(tweets_test[4])

try pour sink bother make taste note 🙂 look like cheap sake taste like ...
0
2/2 [==============================] - 0s 59ms/step
The predicted sentiment is 1
